In [30]:
import selenium as se
from selenium import webdriver

from bs4 import BeautifulSoup

import pandas as pd

**Target:** Scrape data tables on webpage


In [31]:
target_url = 'https://www.cfr.org/backgrounder/economics-hosting-olympic-games'

driver = webdriver.Chrome()

driver.get(target_url)

html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

# Save soup to file
with open('soup.html', 'w') as f:
    f.write(soup.prettify())

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.182)
Stacktrace:
0   chromedriver                        0x0000000103112a0c chromedriver + 4385292
1   chromedriver                        0x000000010310b318 chromedriver + 4354840
2   chromedriver                        0x0000000102d28b0c chromedriver + 281356
3   chromedriver                        0x0000000102d03af0 chromedriver + 129776
4   chromedriver                        0x0000000102d90314 chromedriver + 705300
5   chromedriver                        0x0000000102da3438 chromedriver + 783416
6   chromedriver                        0x0000000102d5feec chromedriver + 507628
7   chromedriver                        0x0000000102d608c4 chromedriver + 510148
8   chromedriver                        0x00000001030da3c8 chromedriver + 4154312
9   chromedriver                        0x00000001030dee2c chromedriver + 4173356
10  chromedriver  

1. Scrape table `The Olympics Have Never Come In Under Budget`

- The data table doesn't appear to be within the saved page HTML.
- Why? The table is loaded in as an iframe, so we won't be able to see it in the HTML using normal `driver.page_source` methods.
- The outer HTML element for the iframe is:
```html
<iframe title="The Olympics Have Never Come in Under Budget" aria-label="Table" id="datawrapper-chart-cUV3B" src="https://datawrapper.dwcdn.net/cUV3B/5/" scrolling="no" frameborder="0" style="width: 0px; border: none; min-width: 100% !important; height: 800px;" height="751" data-external="1">
    ...

</iframe>
```

- By default the selenium.webdriver object is set to the default page which it has parsed. To get the iframe data you will have to switch to the given iframe. Source: [Stackoverflow](https://stackoverflow.com/questions/49825722/scraping-iframe-using-selenium)

In [ ]:
target_url = 'https://www.cfr.org/backgrounder/economics-hosting-olympic-games'

# Open the Chrome browser
driver = webdriver.Chrome()

# Open the target URL
driver.get(target_url)

# Find the <iframe> tag with the attribute id="datawrapper-chart-cUV3B"
frame = driver.find_element("xpath", "//iframe[@id='datawrapper-chart-cUV3B']")

# switch the webdriver object to the iframe.
driver.switch_to.frame(frame)

# Get the page html
html = driver.page_source
# Parse the html using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Save soup to file, so we can search through to check if we have the correct data
with open('soup_iframe.html', 'w') as f:
    f.write(soup.prettify())

# Close the driver
driver.quit()

Searching the saved HTML for some table contents shows we are now successfully retrieving the table data. 

- Next, need to find the Table using BeautifulSoup and build a DataFrame from its contents.

In [ ]:
# Get table in soup
table = soup.find('table')
table

<table cellspacing="0" class="medium datawrapper-cUV3B-1k2zto9 svelte-1smti70 compact resortable"><caption class="sr-only">Table with 5 columns and 19 rows.</caption><thead><tr class="datawrapper-cUV3B-1wbuely svelte-1smti70"><th class="first type-date align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="Year" data-row="-1" scope="col" style="width: 3%;min-width: 10px;border-bottom:1px solid #4B535D;color: #04284A;font-size: 100%;"><button class="sort svelte-1smti70"><span class="content dw-bold">Year</span></button></th><th class="type-text align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="City" data-row="-1" scope="col" style="width: 21%;min-width: 27px;border-bottom:1px solid #4B535D;color: #04284A;font-size: 100%;"><button class="sort svelte-1smti70"><span class="content dw-bold">City</span></button></th><th class="type-text align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="Season" data-row="-1" scope="col" style="width: 3%;min-wid

In [ ]:
# In the table, get the table rows
rows = table.find_all('tr')
rows

[<tr class="datawrapper-cUV3B-1wbuely svelte-1smti70"><th class="first type-date align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="Year" data-row="-1" scope="col" style="width: 3%;min-width: 10px;border-bottom:1px solid #4B535D;color: #04284A;font-size: 100%;"><button class="sort svelte-1smti70"><span class="content dw-bold">Year</span></button></th><th class="type-text align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="City" data-row="-1" scope="col" style="width: 21%;min-width: 27px;border-bottom:1px solid #4B535D;color: #04284A;font-size: 100%;"><button class="sort svelte-1smti70"><span class="content dw-bold">City</span></button></th><th class="type-text align-left resortable cell svelte-1fhgpfc" colspan="1" data-column="Season" data-row="-1" scope="col" style="width: 3%;min-width: 10px;border-bottom:1px solid #4B535D;color: #04284A;font-size: 100%;"><button class="sort svelte-1smti70"><span class="content dw-bold">Season</span></button></th><th c

The first row looks like it contains the column headers, so we can extract these separately.

In [ ]:
# Get table headers
headers = [header.get_text(strip=True) for header in rows[0].find_all('th')]
print(headers)

['Year', 'City', 'Season', 'Cost', 'Overrun']


**Note:**

```python
    headers = [header.get_text() for header in rows[0].find_all('th')]
```
Is equivalent to:

```python
    headers = []
    for header in rows[0].find_all('th'):
        headers.append(header.get_text())
```

<br>
<br>

Next, we'll try and get the table data (from row 1 onwards):
- In each row (i.e. element starting with the `tr` tag), there is 5 child elements, one for each column.
- The first child element is a `th` tag and it looks like this contains the `Year` value.
- The next four elements are `td` tags, and they contain values for `City`, `Season`, `Cost`, and `Overrun`.

In [ ]:
# Test get the value from the first row and first column (should be the Year value).
    # Note: since there is only one `th` tag in each row, we can use the `find` method instead of `find_all`
rows[1].find('th').get_text(strip=True)

'1988'

In [ ]:
# Test getting the values for the remaining columns in the first row
rows[1].find_all('td')

[<td class="type-text dw-bold align-left align-vertical-middle td cell svelte-1ws307v" colspan="1" style="font-size: 100%; width: 21%; min-width: 27px;">Calgary</td>,
 <td class="type-text align-left align-vertical-middle td cell svelte-1ws307v" colspan="1" style="color: rgb(158, 163, 165); font-size: 90%; width: 3%; min-width: 10px;">Winter</td>,
 <td class="type-bar align-left align-vertical-middle td cell svelte-1ws307v" colspan="1" style="font-size: 100%; width: 66%; min-width: 70px;"><div class="sr-only">$1.2B</div><div aria-hidden="true" class="bar-chart datawrapper-cUV3B-1udwhp8 svelte-enpqx1 no-background normal" title="$1.2B"><div class="baseline svelte-enpqx1" style="display: none"></div><div class="bar svelte-enpqx1" style="background: #2e75b8;margin-left: 0%;width: 4.1522491349480966%"><div class="bar-label invisible svelte-enpqx1">$1.2B</div><div class="bar-label svelte-enpqx1" style="left: 4.1522491349480966%;width: 95.8477508650519%;text-align: left;padding-left: 3px;ove

In [ ]:
# Test extracting text from each item in list of td tags
[td.text for td in rows[1].find_all('td')]

['Calgary', 'Winter', '$1.2B$1.2B$1.2B', '65%']

The values in the `Cost` column are repeated, so there might be more child elements for this column, which is causing the repeated text. 
- let's use a normal list iteration to account for this.

In [ ]:
data = []
for td in rows[1].find_all('td'):
    data.append(td.text)

data

['Calgary', 'Winter', '$1.2B$1.2B$1.2B', '65%']

Let's look at the html code for the element in the row containing the `Cost` values. 

```html
<td colspan="1" class="type-bar align-left align-vertical-middle td cell svelte-1ws307v" style="font-size: 100%; width: 66%; min-width: 70px;">
    <div class="sr-only">$1.2B</div>
    <div title="$1.2B" class="bar-chart datawrapper-cUV3B-1udwhp8 svelte-enpqx1 no-background normal" aria-hidden="true">
        <div class="baseline svelte-enpqx1" style="display: none"></div>
        <div class="bar svelte-enpqx1" style="background: #2e75b8;margin-left: 0%;width: 4.1522491349480966%">
            <div class="bar-label invisible svelte-enpqx1">$1.2B</div>
            <div style="left: 4.1522491349480966%;width: 95.8477508650519%;text-align: left;padding-left: 3px;overflow: hidden;white-space: nowrap;;text-overflow: ellipsis;" class="bar-label svelte-enpqx1">$1.2B</div>
        </div>
    </div>
</td>
```

- We could add a condition to our list iteration to check if the element corresponds to the `Cost` column.

In [ ]:
data = []

for td in rows[1].find_all('td'):
    # Add condition to check if td corresponds to `Cost` element. 
    if td.find('div', class_='sr-only'):        # Checks existence of div tag with class 'sr-only', if yes, then it is the 'Cost' element
        # Get the text from a child div tag
        data.append(td.find('div', class_='sr-only').get_text())
    else: # Other elements, just get the text from the td tag
        data.append(td.get_text())

print(data)

['Calgary', 'Winter', '$1.2B', '65%']


<br>

Finally, need to put this together to apply for every row. 

In [ ]:
data = []

# Loop through all rows in the table
for row in rows:
    row_data = []
    for td in row.find_all('td'):
        # Add condition to check if td corresponds to `Cost` element.
        if td.find('div', class_='sr-only'):
            # Get the text from a child div tag
            row_data.append(td.find('div', class_='sr-only').get_text())
        else:
            # Other elements, just get the text from the td tag
            row_data.append(td.get_text())
    data.append(row_data)

# Print the collected data
for row in data:
    print(row)


[]
['Calgary', 'Winter', '$1.2B', '65%']
['Albertville', 'Winter', '$2.1B', '137%']
['Barcelona', 'Summer', '$11.6B', '266%']
['Lillehammer', 'Winter', '$3.4B', '277%']
['Atlanta', 'Summer', '$4.7B', '151%']
['Nagano', 'Winter', '$2.2B', '56%']
['Sydney', 'Summer', '$5.2B', '90%']
['Salt Lake City', 'Winter', '$2.7B', '24%']
['Athens', 'Summer', '$3.1B', '49%']
['Turin', 'Winter', '$4.7B', '80%']
['Beijing', 'Summer', '$8.3B', '2%']
['Vancouver', 'Winter', '$3.2B', '13%']
['London', 'Summer', '$16.8B', '76%']
['Sochi', 'Winter', '$28.9B', '289%']
['Rio de Janeiro', 'Summer', '$23.6B', '352%']
['Pyeongchang', 'Winter', '$3.4B', '2%']
['Tokyo', 'Summer', '$13.7B', '128%']
['Beijing', 'Winter', '$8.7B', '149%']
['Paris', 'Summer', '$8.7B', '115%']


In [ ]:
data = []


# Loop through all rows in the table
for row in rows:
    row_data = []
    # Extract text from the first `th` element in the first row
    if row.find('th'):
        row_data.append(row.find('th').get_text(strip=True))
    # Extract text from `td` elements in the subsequent rows
    for td in row.find_all('td'):
        if td.find('div', class_='sr-only'):
            # Get the text from a child div tag
            row_data.append(td.find('div', class_='sr-only').get_text())
        else:
            # Other elements, just get the text from the td tag
            row_data.append(td.get_text())
    data.append(row_data)

# Print the collected data
for row in data:
    print(row)



['Year']
['1988', 'Calgary', 'Winter', '$1.2B', '65%']
['1992', 'Albertville', 'Winter', '$2.1B', '137%']
['1992', 'Barcelona', 'Summer', '$11.6B', '266%']
['1994', 'Lillehammer', 'Winter', '$3.4B', '277%']
['1996', 'Atlanta', 'Summer', '$4.7B', '151%']
['1998', 'Nagano', 'Winter', '$2.2B', '56%']
['2000', 'Sydney', 'Summer', '$5.2B', '90%']
['2002', 'Salt Lake City', 'Winter', '$2.7B', '24%']
['2004', 'Athens', 'Summer', '$3.1B', '49%']
['2006', 'Turin', 'Winter', '$4.7B', '80%']
['2008', 'Beijing', 'Summer', '$8.3B', '2%']
['2010', 'Vancouver', 'Winter', '$3.2B', '13%']
['2012', 'London', 'Summer', '$16.8B', '76%']
['2014', 'Sochi', 'Winter', '$28.9B', '289%']
['2016', 'Rio de Janeiro', 'Summer', '$23.6B', '352%']
['2018', 'Pyeongchang', 'Winter', '$3.4B', '2%']
['2020', 'Tokyo', 'Summer', '$13.7B', '128%']
['2022', 'Beijing', 'Winter', '$8.7B', '149%']
['2024', 'Paris', 'Summer', '$8.7B', '115%']


In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

,0,1,2,3,4
0,Year,None,None,None,None
1,1988,Calgary,Winter,$1.2B,65%
2,1992,Albertville,Winter,$2.1B,137%
3,1992,Barcelona,Summer,$11.6B,266%
4,1994,Lillehammer,Winter,$3.4B,277%
5,1996,Atlanta,Summer,$4.7B,151%
6,1998,Nagano,Winter,$2.2B,56%
7,2000,Sydney,Summer,$5.2B,90%
8,2002,Salt Lake City,Winter,$2.7B,24%
9,2004,Athens,Summer,$3.1B,49%


In [ ]:
df2 = df.drop(0).reset_index(drop=True)

# Display the DataFrame
df2

,0,1,2,3,4
0,1988,Calgary,Winter,$1.2B,65%
1,1992,Albertville,Winter,$2.1B,137%
2,1992,Barcelona,Summer,$11.6B,266%
3,1994,Lillehammer,Winter,$3.4B,277%
4,1996,Atlanta,Summer,$4.7B,151%
5,1998,Nagano,Winter,$2.2B,56%
6,2000,Sydney,Summer,$5.2B,90%
7,2002,Salt Lake City,Winter,$2.7B,24%
8,2004,Athens,Summer,$3.1B,49%
9,2006,Turin,Winter,$4.7B,80%


In [ ]:
df2.columns = ['year', 'city', 'season', 'cost', 'overrun']

# Display the DataFrame
print(df2)

    year            city  season    cost overrun
0   1988         Calgary  Winter   $1.2B     65%
1   1992     Albertville  Winter   $2.1B    137%
2   1992       Barcelona  Summer  $11.6B    266%
3   1994     Lillehammer  Winter   $3.4B    277%
4   1996         Atlanta  Summer   $4.7B    151%
5   1998          Nagano  Winter   $2.2B     56%
6   2000          Sydney  Summer   $5.2B     90%
7   2002  Salt Lake City  Winter   $2.7B     24%
8   2004          Athens  Summer   $3.1B     49%
9   2006           Turin  Winter   $4.7B     80%
10  2008         Beijing  Summer   $8.3B      2%
11  2010       Vancouver  Winter   $3.2B     13%
12  2012          London  Summer  $16.8B     76%
13  2014           Sochi  Winter  $28.9B    289%
14  2016  Rio de Janeiro  Summer  $23.6B    352%
15  2018     Pyeongchang  Winter   $3.4B      2%
16  2020           Tokyo  Summer  $13.7B    128%
17  2022         Beijing  Winter   $8.7B    149%
18  2024           Paris  Summer   $8.7B    115%


In [ ]:
import csv
csv_file = 'olympics_costs.csv'

# Open the file in write mode
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write each row from the list to the CSV file
    writer.writerows(data)

print(f"Data has been written to {csv_file}")

Data has been written to olympics_costs.csv


In [34]:
df2['cost'] = df2['cost'].str.replace('%', '', regex=False).str.replace('$', '', regex=False)
df2['overrun'] = df2['overrun'].str.replace('%', '', regex=False).str.replace('$', '', regex=False)

# Display the DataFrame
print(df2)

    year            city  season  cost overrun
0   1988         Calgary  Winter   1.2      65
1   1992     Albertville  Winter   2.1     137
2   1992       Barcelona  Summer  11.6     266
3   1994     Lillehammer  Winter   3.4     277
4   1996         Atlanta  Summer   4.7     151
5   1998          Nagano  Winter   2.2      56
6   2000          Sydney  Summer   5.2      90
7   2002  Salt Lake City  Winter   2.7      24
8   2004          Athens  Summer   3.1      49
9   2006           Turin  Winter   4.7      80
10  2008         Beijing  Summer   8.3       2
11  2010       Vancouver  Winter   3.2      13
12  2012          London  Summer  16.8      76
13  2014           Sochi  Winter  28.9     289
14  2016  Rio de Janeiro  Summer  23.6     352
15  2018     Pyeongchang  Winter   3.4       2
16  2020           Tokyo  Summer  13.7     128
17  2022         Beijing  Winter   8.7     149
18  2024           Paris  Summer   8.7     115


In [37]:
import json

olympic_costs = list(df2.T.to_dict().values())

open('olympic_costs.json','w').write(json.dumps(olympic_costs))


1721